In [1]:
import subprocess
from pydub import AudioSegment
import math
import openai
from dotenv import load_dotenv
import os
import glob

c:\Users\ppjjh\anaconda3\envs\GPT\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [19]:
def extract_audio_from_video(video_path, audio_path):
    command = ["ffmpeg", "-i",video_path, "-vn", 
    audio_path,]
    subprocess.run(command)

def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000
    chunks = math.ceil(len(track)/chunk_len)
    for i in range(chunks):
        start_time = i*chunk_len
        end_time = (i+1) * chunk_len
        chunk = track[start_time:end_time]
        chunk.export(f"{chunks_folder}/chunk_{i}.mp3", format= "mp3")
    

In [ ]:
extract_audio_from_video("./files/jocoding_ai_news.mp4", "files/audio.mp3")

In [21]:
cut_audio_in_chunks("./files/audio.mp3", 10, "./files/chunks")

In [2]:

# .env 파일에서 환경변수 불러오기
load_dotenv()

# 환경변수에서 API 키 가져오기
openai.api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    final_transcript = ""
    for file in files :
        with open(file, "rb") as audio_file:
            transcript = openai.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            language="ko"
            )
            final_transcript += transcript.text
    with open(destination, "w") as file:
        file.write(final_transcript)

# # 기존것에 더하는 함수
# def transcribe_chunks(chunk_folder, destination):
#     files = glob.glob(f"{chunk_folder}/*.mp3")
#     # final_transcript = ""
#     for file in files :
#         with open(file, "rb") as audio_file, open(destination,"a") as text_file:
#             transcript = openai.audio.transcriptions.create(
#             model="whisper-1",
#             file=audio_file,
#             language="ko"
#             )
#             text_file.write(transcript.text)
    

In [4]:
transcribe_chunks("./files/chunks/", "./files/transcript.txt")